# Nuclei Segmentation

Let's begin with nuclei segmentation.
We can either segment the DAPI channel and then filter out the healthy nuclei, segment only the healthy nuclei or go for the ISL1 channel that has the label for the nuclei of interest.

Let's select an image and then crop out a small piece of it to make testing easier.

In [1]:
from pathlib import Path

import numpy as np
from czifile import CziFile
from napari import Viewer
from skimage.morphology import ball, erosion, remove_small_objects

In [2]:
DATA_DIR = Path(
    "../data/20240729/980/20x_with_z_correction/NS#4_Healthy_5uM_MI132_D19_G3BP,Iselt,DAPI_20x+ZCorr.czi"
)
image_handle = CziFile(DATA_DIR)
img = np.squeeze(image_handle.asarray())[2, :, 1250:2000, 1250:2000]

In [3]:
scale = {
    values_dict["Id"]: values_dict["Value"] * 10**6
    for values_dict in image_handle.metadata(raw=False)["ImageDocument"]["Metadata"][
        "Scaling"
    ]["Items"]["Distance"]
}
spacing = (scale["Z"], scale["Y"], scale["X"])
spacing

(0.75, 0.18130982905982898, 0.18130982905982898)

## Preprocessing image

We will reduce noise in the image by applying a gaussian filter to it.

In [4]:
from skimage.filters import gaussian # noqa

In [ ]:
smoothed_img = gaussian(img, sigma=(1/spacing[0], 0.5/spacing[1], 0.5/spacing[2]))

In [6]:
viewer = Viewer()
viewer.add_image(img, scale=spacing, visible=False)
viewer.add_image(smoothed_img, scale=spacing, visible=True)

<Image layer 'smoothed_img' at 0x24f02ea0ef0>

## Cellpose

Let's begin by trying out cellpose.

In [7]:
from cellpose import models  # noqa
from cellpose.io import logger_setup  # noqa

logger_setup()

2024-11-28 13:55:25,209 [INFO] WRITING LOG OUTPUT TO C:\Users\aguco599\.cellpose\run.log
2024-11-28 13:55:25,210 [INFO] 
cellpose version: 	3.1.0 
platform:       	win32 
python version: 	3.12.7 
torch version:  	2.2.2


(<Logger cellpose.io (INFO)>,
 WindowsPath('C:/Users/aguco599/.cellpose/run.log'))

In [8]:
model = models.Cellpose(model_type="cyto3")

2024-11-28 13:55:25,255 [INFO] >>>> using CPU
2024-11-28 13:55:25,256 [INFO] >>>> using CPU
2024-11-28 13:55:25,256 [INFO] >> cyto3 << model set to be used
2024-11-28 13:55:25,294 [INFO] >>>> loading model C:\Users\aguco599\.cellpose\models\cyto3
2024-11-28 13:55:25,375 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


After a couple attempts with several algorithms, I thought there might be issues with all the saturated nuclei.
To circunvent this, I made a mask for those pixels, calculated the distance transform and added this to the original image in order to have some values here.

In [9]:
anisotropy = scale["Z"] / scale["X"]
print(anisotropy)

4.136565589902539


In [10]:
diameter = 10 / scale["X"]
print(diameter)

55.15420786536719


In [11]:
masks, flows, styles, diams = model.eval(
    [smoothed_img],
    batch_size=8,
    diameter=diameter,
    z_axis=0,
    do_3D=True,
    channels=[0, 0],
    # stitch_threshold=0.5,
    normalize=True,
    flow_threshold=0.2,
    cellprob_threshold=4.0,
    anisotropy=anisotropy,
)

2024-11-28 13:55:25,449 [INFO] channels set to [0, 0]
2024-11-28 13:55:25,449 [INFO] ~~~ FINDING MASKS ~~~
2024-11-28 13:55:25,450 [INFO] multi-stack tiff read in as having 74 planes 1 channels
2024-11-28 13:55:25,813 [INFO] resizing 3D image with rescale=0.54 and anisotropy=4.136565589902539
2024-11-28 13:55:25,960 [INFO] running YX: 166 planes of size (407, 407)
2024-11-28 13:57:37,243 [INFO] 100%|##########| 166/166 [02:11<00:00,  1.26it/s]
2024-11-28 13:57:37,447 [INFO] running ZY: 407 planes of size (166, 407)
2024-11-28 13:59:01,964 [INFO] 100%|##########| 204/204 [01:24<00:00,  2.41it/s]
2024-11-28 13:59:02,174 [INFO] running ZX: 407 planes of size (166, 407)
2024-11-28 14:00:24,771 [INFO] 100%|##########| 204/204 [01:22<00:00,  2.47it/s]
2024-11-28 14:00:25,121 [INFO] resizing 3D flows and cellprob to original image size
2024-11-28 14:00:26,128 [INFO] network run in 300.32s
2024-11-28 14:03:10,633 [INFO] masks created in 164.32s
2024-11-28 14:03:13,087 [INFO] >>>> TOTAL TIME 46

In [12]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(masks[0], scale=spacing)

<Labels layer 'Labels' at 0x24f2a2cb020>

It's working quite well for most of the nuclei.
Some are still over segmented and this needs some filtering of small objects.

In [13]:
voxel_size = scale["X"] * scale["Y"] * scale["Z"]
print(f"voxel size is {voxel_size}")
minimum_nuclei_volume = 2000 * voxel_size
print(
    f"The minimum size for an acceptable nuclei "
    f"size would be {minimum_nuclei_volume} um3"
)

voxel size is 0.024654940585278305
The minimum size for an acceptable nuclei size would be 49.30988117055661 um3


In [14]:
from skimage.util import apply_parallel  # noqa

label = apply_parallel(erosion, masks[0], chunks=(1, masks[0].shape[1], masks[0].shape[2]), compute=True, extra_keywords={"footprint": ball(4)}, dtype="uint16")
label = remove_small_objects(label, minimum_nuclei_volume / voxel_size)

In [15]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(label, scale=spacing)

<Labels layer 'label' at 0x24f13def4a0>

After this smoothing of the labels, segmentation of nuclei looks quite fine.